<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/jupyter/chains/sequential_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

In [2]:
! pip show langchain

Name: langchain
Version: 0.3.19
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/eb1899961595710f0d6f79f7c530aa5d8cf704eb/langchain/jupyter/images/llm_chain.png?raw=1)

In [4]:
! pip install -U langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00


In [9]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

# llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500,
    api_key="",
    base_url="https://vip.apiyi.com/v1"
)

In [10]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [16]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
# print(chain.invoke({
#     'product': "性能卓越的GPU"
#     }))

# 这将在 langchain 处理 LLM 的响应之前直接打印它，
# 从而让您可以看到 LLM 实际返回的内容。
raw_response = llm(prompt.format(product="性能卓越的GPU"))
print(f"Raw LLM response: {raw_response}")

print(chain.invoke({
    'product': "性能卓越的GPU"
    }))


Raw LLM response: 

1. 翼龙威视（Eagle Vision Technologies Co., Ltd.）
2. 光速梦想（LightSpeed Dreams Co., Ltd.）
3. 极像科技（Ultiimage Technologies Co., Ltd.）
4. 创威云图（PowerCloud Graphics Co., Ltd.）
5. 无限炽火（Infinite Inferno Co., Ltd.）
6. 神威强化（Divine Strength Co., Ltd.）
7. 飞溅光影（Splash of Light Co., Ltd.）
8. 色环智控（Color Circle Intelligence Co., Ltd.）
9. 量子视界（Quantum Vision Co., Ltd.）
10. 星域璀璨（Galactic Brilliance Co., Ltd.）
{'product': '性能卓越的GPU', 'text': "\n\n1．蓝宝石图形处理器有限公司（Sapphire Graphics Limited Company）\n2．星辰科技图形处理器有限公司（StellarTech Graphics Limited Company）\n3．火焰骑士图形处理器有限公司（FlameKnight Graphics Limited Company）\n4．钻石视界图形处理器有限公司（DiamondVision Graphics Limited Company）\n5．虚幻之美图形处理器有限公司（Unreal Beauty Graphics Limited Company）\n6．极速进化图形处理器有限公司（Rapid Evolution Graphics Limited Company）\n7．魔法师工作室图形处理器有限公司（Wizard's Workshop Graphics Limited Company）\n8．冰川科技图形处理器有限公司（GlacierTech Graphics Limited Company）\n9．黑曜石精英图形处理器有限公司（Obsidian Elite Graphics Limited Company）\n10．天幕视觉图形处理器有限公司（SkyVision Graphics Limited 

In [17]:
chain.verbose =True

In [18]:
chain.verbose

True

In [20]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
{'product': '性能卓越的GPU', 'text': '\n\n1. 铸星科技有限公司 (CastTech Co., Ltd.)\n2. 极力图形有限公司 (PixForce Graphics, Ltd.)\n3. 灵动GPU有限公司 (DynamicGPU Co., Ltd.)\n4. 心源科技有限公司 (HeartCore Technology Co., Ltd.)\n5. 范诺斯特科技有限公司 (Fenonost Technology Co., Ltd.)\n6. 虚拟极限科技有限公司 (Virtual Limits Tech Co., Ltd.)\n7. 集创图形有限公司 (UniteGraphics Co., Ltd.)\n8. 瞬变科技有限公司 (TransientTech Co., Ltd.)\n9. 幻境计算有限公司 (Illusionary Computing Co., Ltd.)\n10. 博景科技有限公司 (Bojie Technology Co., Ltd.)'}


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/eb1899961595710f0d6f79f7c530aa5d8cf704eb/langchain/jupyter/images/simple_sequential_chain_0.png?raw=1)

In [22]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

# llm = OpenAI(temperature=0.7, max_tokens=1000)
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct", temperature=0.7, max_tokens=1000,
    api_key="",
    base_url="https://vip.apiyi.com/v1"
)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [23]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/eb1899961595710f0d6f79f7c530aa5d8cf704eb/langchain/jupyter/images/simple_sequential_chain_1.png?raw=1)

In [24]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [25]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...

《三体人不是无法战胜的》是一部关于勇气和坚持的戏剧作品。故事发生在一个遥远的宇宙中，一个充满危险和挑战的星球上。这个星球上的居民是三体人，他们拥有强大的科技和强大的战斗力，被认为是无法战胜的存在。

然而，一群勇敢的人类冒着生命危险，踏上了挑战三体人的征程。经历了无数次的失败和挫折，他们终于发现了三体人的弱点，也发现了他们并非无敌。在无数的牺牲和努力下，人类终于战胜了三体人，拯救了自己的家园。

这部戏剧充满了惊险和感动的场景，展现了人类的勇气和坚持不懈的精神。它也提醒我们，面对困难和挑战，只要我们不放弃，就没有什么是无法战胜的。让我们一起观看《三体人不是无法战胜的》，领略勇气和希望的力量。


《三体人不是无法战胜的》是一部令人叹为观止的戏剧作品，它为我们展现了勇气和坚持的力量。故事发生在一个遥远的宇宙中，充满了危险和挑战的星球上。这个星球上的居民是强大的三体人，他们拥有令人惊叹的科技和战斗力，被认为是无敌的存在。

然而，一群勇敢的人类冒着生命危险，决心挑战三体人。他们经历了无数次的失败和挫折，但却始终坚持不懈，最终发现了三体人的弱点。在无数次的牺牲和努力下，人类终于战胜了三体人，拯救了自己的家园。

这部戏剧充满了惊险和感动的场景，让观众们跟随着人类的冒险旅程，体验着他们的艰辛和胜利。它也让我们反思，面对困难和挑战，我们是否也能像剧中的人物一样坚持不懈，最终战胜困难？这部作品也提醒我们，只要我们有勇气和希望，就没有什么是无法战胜的。

演员们的精湛表演和精彩的舞台布景，让这部戏剧更加生动和震撼。导演的精心安排和编排，也让剧情紧凑而又引人入胜。观众们在欢笑和泪水中，一起见证了人类的勇气和坚持的伟大力量。

总的来说，《三体人不是无法战胜的》是一部值得一看的作品，它不仅仅是一部娱乐性的戏剧，更是一部让我们反思和感悟的作品。让我们一起走进这个遥远的宇宙，领略勇气和希望的力量。

> Finished chain.


In [28]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满冒险和惊险的科幻戏剧。故事发生在遥远的未来，人类和外星种族之间的战争仍在继续。在上一部戏剧中，反抗军成功打败了帝国，但是新的威胁出现了。一群名为“第一秩序”的组织试图重建帝国的统治，而反抗军必须再次团结起来，与他们作斗争。与此同时，一位年轻的女孩，名叫雷伊，发现自己具有强大的原力，她必须学习控制和运用这种力量，来帮助反抗军打败第一秩序。在这场决战中，古老的敌人和新的盟友都会出现，每个人都将为了自己的信念而战斗。《星球大战第九季》将展现出宏大的场景、精彩的战斗以及角色之间的情感纠葛，带领观众进入一场精彩绝伦的星际冒险之旅。


《星球大战第九季》是一部令人叹为观止的科幻巨制。导演将观众带入了一个充满惊险和冒险的遥远未来世界，让我们再次见证人类与外星种族之间的战争。从上一部戏剧中的胜利，到现在的新威胁，故事情节跌宕起伏，令人心跳加速。

这部戏剧不仅有精彩的战斗场面，更展现了角色之间的情感纠葛。年轻的女主角雷伊展现出了强大的原力，她必须学习控制并运用这种力量来帮助反抗军打败第一秩序。她的成长过程也让观众在战斗中感受到了深刻的人性冲突和成长。

一如既往，《星球大战第九季》也带来了宏大的场景和精彩的特效。观众将被带入一个充满绚丽色彩和惊险刺激的星际冒险世界，每一场战斗都令人热血沸腾。

除了经典的角色外，新的盟友和古老的敌人也将在这场决战中出现。每个人都为了自己的信念而战斗，让观众在剧情的高潮中感受到了强烈的情感共鸣。

总的来说，《星球大战第九季》是一部不容错过的科幻巨制。它将带领观众进入一个充满惊险和冒险的世界，让人们再次感受到了星球大战系列带来的震撼和感动。无论是《星球大战》的粉丝还是科幻电影爱好者，都不应该错过这部精彩绝伦的戏剧。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/eb1899961595710f0d6f79f7c530aa5d8cf704eb/langchain/jupyter/images/sequential_chain_0.png?raw=1)

In [49]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct", temperature=0.7, max_tokens=1000,
    api_key="",
    base_url="https://vip.apiyi.com/v1"
)

# 智谱API调用(glm-4v-flash)
# llm = OpenAI(
#     model_name="glm-4v-flash", temperature=0.7, max_tokens=1000,
#     api_key="XXX",
#     base_url="https://open.bigmodel.cn/api/paas/v4/chat"
# )


template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [50]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [51]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [61]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《三体人不是无法战胜的》是一部以二十一世纪的新中国为背景的戏剧作品。在这个时代，人类社会已经取得了巨大的科技进步，但同时也面临着前所未有的挑战。在与外星文明三体人的战争中，人类陷入了绝境。但是，主人公李明并不认为三体人是无法战胜的。他不断探索新的战略和技术，带领人类抵抗外星入侵，并最终取得了胜利。

这部戏剧将展现李明和其他勇敢的科学家、战士们在生死关头的奋斗，同时也揭示出人类在面对危机时的勇气和坚持。通过戏剧的形式，观众将感受到科技与人性的冲突，以及人类面对未知挑战时的自信和坚定。同时，这部作品也将提出对于科技发展与人类未来的深刻思考，引发观众对于未来的反思。

《三体人不是无法战胜的》既是一部科幻戏剧，也是一部关于人类信念和勇气的故事。它将带领观众穿越时空，探索人类的未来，为观众带来震撼与感动。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n《三体人不是无法战胜的》是一部以二十一世纪的新中国为背景的戏剧作品。在这个时代，人类社会已经取得了巨大的科技进步，但同时也面临着前所未有的挑战。在与外星文明三体人的战争中，人类陷入了绝境。但是，主人公李明并不认为三体人是无法战胜的。他不断探索新的战略和技术，带领人类抵抗外星入侵，并最终取得了胜利。\n\n这部戏剧将展现李明和其他勇敢的科学家、战士们在生死关头的奋斗，同时也揭示出人类在面对危机时的勇气和坚持。通过戏剧的形式，观众将感受到科技与人性的冲突，以及人类面对未知挑战时的自信和坚定。同时，这部作品也将提出对于科技发展与人类未来的深刻思考，引发观众对于未来的反思。\n\n《三体人不是无法战胜的》既是一部科幻戏剧，也是一部关于人类信念和勇气的故事。它将带领观众穿越时空，探索人类的未来，为观众带来震撼与感动。',
 'review': ' \n\n《三体人不是无法战胜的》是一部令人兴奋的戏剧作品，它将人类的科技进步与外星文明的入侵相结合，展现出一场惊心动魄的战争。剧中主人公李明的勇气和决心令人钦佩，他的不懈努力和聪明智慧带领着人类走出绝境，最终取得了胜利。\n\n这部戏剧并非仅仅是一场关于科技与外星战争的故事，它更是一次对人类信念和勇气的探讨。在面对前所未有的危机时，人类并没有放弃，而是勇敢地面对挑战，最终战胜了强大的敌人。这种人性的力量令人动容，也让观众深思。\n\n此外，剧中也提出了对于科技发展和人类未来的深刻思考。在人类取得巨大科技进步的同时，也面临着可能带来的危险和挑战。这部作品引发观众对于未来的反思，也让我们意识到科技发展需要与人类的信念和价值观相结合，才能更好地面对未来。\n\n最后，这部戏剧的制作也令人惊叹。精彩的舞台布景和逼真的特效为观众带来身临其境的感受，加上演员们出色的表演，让观众完全沉浸于剧情中。《三体人不是无法战胜的》不仅是一部科幻作品，更是一场视觉盛宴。\n\n总的来说，《三体人不是无法战胜的》是一部充满惊喜和感动的戏剧，它将带领观众探索人类的未来，同时也提出了对于科技发展和人类信念的深刻思考。我强烈推荐观众们去体验这场震撼人心的戏剧。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果